In [28]:
import numpy as np
import pandas as pd
from models import lstm
import importlib
import torch

In [13]:
input_dim = 39
hidden = 512
languages = 2
snippet_length = 75
batch size = 10

In [15]:
data_path = '/Users/oosv/data/csv'
data_en = pd.read_csv(f"{data_path}/english_data.csv").values.astype(np.float32)
data_es = pd.read_csv(f"{data_path}/spanish_data.csv").values.astype(np.float32)
print(np.shape(data_en))
print(np.shape(data_es))

(4939049, 39)
(1760399, 39)


In [17]:
# shape data in 3 dimensions and prepare for training
data_en_sampled = np.reshape(data_en[74:], (-1,75,39))
data_es_sampled = np.reshape(data_es[74:], (-1,75,39))
en_idx = int(len(data_en_sampled)/4)
es_idx = int(len(data_es_sampled)/4)

# Training models on equal amounts of data
train_en = data_en_sampled[es_idx:len(data_es_sampled)]
train_es = data_es_sampled[es_idx:]
test_en = data_en_sampled[:es_idx]
test_es = data_en_sampled[:es_idx]

In [18]:
print(np.shape(train_en), np.shape(train_es))
print(np.shape(test_en), np.shape(test_es))

(17604, 75, 39) (17604, 75, 39)
(5867, 75, 39) (5867, 75, 39)


In [101]:
#train_data = np.concatenate((train_en, train_es), axis = 0)
shape_es = np.shape(train_en)
shape_en = np.shape(train_es)
train_labels = np.concatenate((np.zeros((shape_en[0],shape_en[1],1)), np.ones((shape_es[0],shape_es[1],1))), axis = 0)
print(np.shape(train_data))
print(np.shape(train_labels))

(35208, 75, 39)
(35208, 75, 1)


In [102]:
def shuffle_in_unison(a, b):
    rng_state = np.random.get_state()
    np.random.shuffle(a)
    np.random.set_state(rng_state)
    np.random.shuffle(b)

In [320]:
print(train_data[0][0])
shuffle_in_unison(train_data, train_labels)
print(train_data[0][0])

[ 9.4133883e+00 -1.9497540e+01  3.9959562e+00  5.6146326e+00
 -6.8843942e+00  1.2543772e+01 -1.0601465e+00 -1.8551652e+00
  3.9233890e-01 -8.9237595e+00  1.3722342e+00 -2.4326236e+00
 -2.9970469e+00 -6.0930759e-02  3.9699584e-02 -2.6023310e-02
 -3.0821905e-01  9.3048459e-01  1.2305728e+00 -2.7709448e-01
 -2.6436749e+00 -1.6095033e+00 -3.6033112e-01  5.6367183e-01
  3.8722613e-01  6.2193286e-01  9.1716774e-02 -6.6639298e-01
 -4.6228880e-01  2.3709337e-01 -4.1993576e-01 -3.0728883e-01
 -2.6667345e-01  9.7258455e-01  1.6678059e-02  1.0791121e+00
 -1.1346647e+00  5.6959257e-02 -9.5459871e-02]
[ 16.331465   -15.14927      2.432831   -10.573003    -1.8485426
  14.230032    14.348568    12.307588    -1.3211254   -6.4342237
  12.163789    -7.7444263   10.754949     0.5930134   -2.4643674
   1.3112882   -4.356198    -3.3050075    5.252328    -0.5415032
   0.3599433    0.7070114   -4.9063616    1.5746101   -1.5614756
   2.0184734   -0.16678551   0.59325886  -0.509352     1.6668657
   1.060825   

In [330]:
importlib.reload(lstm)
_lstm = lstm.LSTM()
# _train_data = torch.tensor(train_data)
# _train_labels = torch.tensor(train_labels)
# print(_train_labels.size())

In [ ]:
loss,e_loss = _lstm.train(train_data, train_labels, epoch=2, batch_size = 1000, update=True)

loss: 2.0377087593078613, epoch: 0 iter: 1
loss: 2.0557773113250732, epoch: 0 iter: 2
loss: 2.0946359634399414, epoch: 0 iter: 3
loss: 2.056750774383545, epoch: 0 iter: 4
loss: 2.029942035675049, epoch: 0 iter: 5
loss: 1.9939228296279907, epoch: 0 iter: 6
loss: 1.9740431308746338, epoch: 0 iter: 7
loss: 1.9895633459091187, epoch: 0 iter: 8
loss: 1.9706158638000488, epoch: 0 iter: 9
loss: 1.9679375886917114, epoch: 0 iter: 10
loss: 1.944096326828003, epoch: 0 iter: 11
loss: 1.8979072570800781, epoch: 0 iter: 12
loss: 1.8981534242630005, epoch: 0 iter: 13
loss: 2.005270481109619, epoch: 0 iter: 14
loss: 1.9253475666046143, epoch: 0 iter: 15
loss: 1.9024995565414429, epoch: 0 iter: 16
loss: 1.832659125328064, epoch: 0 iter: 17
loss: 1.8966102600097656, epoch: 0 iter: 18
loss: 1.8803493976593018, epoch: 0 iter: 19
loss: 1.9527474641799927, epoch: 0 iter: 20
loss: 1.8729580640792847, epoch: 0 iter: 21
loss: 1.9012987613677979, epoch: 0 iter: 22
loss: 1.802640438079834, epoch: 0 iter: 23
los

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots()
ax.plot([x for x in range(len(loss))], loss)
plt.show()

In [ ]:
pred = []
with torch.no_grad():
    pred = np.array([np.argmax(_lstm.predict(sample).view(75,2).mean(0)).data for sample in test_en])
    print(np.average(pred == 0))

In [ ]:
pred = []
with torch.no_grad():
    pred = np.array([np.argmax(_lstm.predict(sample).view(75,2).mean(0)).data for sample in test_es])
    print(np.average(pred == 1))

In [260]:

with torch.no_grad():
    print(_lstm.predict(test_en[:5])[:,-1])

tensor([[-4.4987, -4.7444],
        [-3.1931, -3.8124],
        [-3.6228, -3.6808],
        [-4.4007, -4.3718],
        [-3.3266, -3.3327]])


In [ ]:
t = torch.reshape(torch.tensor(test_en[0][0]), (3,13))
print(f'{t[0]} \n')
print(f'{t[1]} \n')
print(f'{t[2]} \n')


In [272]:
r = torch.nn.ReLU()

In [274]:
print(f'{r(t[0])} \n')
print(f'{r(t[1])} \n')
print(f'{r(t[2])} \n')

tensor([10.2047,  0.0000,  0.0000,  0.0000,  2.3345,  1.3560,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000]) 

tensor([0.0136, 0.7787, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.4182, 1.0459, 0.0000]) 

tensor([0.0734, 0.0000, 0.3271, 0.2432, 0.0000, 0.4300, 0.0229, 0.5136, 0.2651,
        0.0000, 0.0000, 0.8104, 0.1049]) 

